<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# La classe Data Frame de <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> [SQL](http://spark.apache.org/sql/)

**Résumé**: Ce tutoriel introduit la classe *data frame* proposée par la librairie [*SparkSQL*](http://spark.apache.org/sql/). Cette classe deviendra un standard pour toutes les manipulations de données structurées à partir de la version 3.0 de *Spark*. 

## 1 Lecture des données
**NB** [*SparkSQL*](http://spark.apache.org/sql/) offre la possibilité de lire et manipuler de très nombreux types de fichiers. Ces fonctionnalités ne sont pas introduites mais elles constituent un atout évident pour justifier du développement de cet environnement.

Ce tutoriel s'inspire de ceux proposés par [J. A. Dianes](https://github.com/jadianes/spark-py-notebooks) pour l'utilisation des données du concours [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) concernant près de 9M d'interactions dans un réseau. Elles sont décrites en détail [ici](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names). L'objectif est d'apprendre à détecter des intrusions dans un réseau à partir d'un ensemble de variables ou *features* déjà calculées sur chaque transaction ou ineraction avec le réseau.

Un sous-échantillon est chargé localement avant de créer la RDD.

In [ ]:
import urllib
f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file).cache()

## 2 Construire un Data Frame

Un *Data Frame* Spark est une collection de données distribuées et organisées en *colonnes* identifiées par des noms. C'est conceptuellement  équivalent à une table dans une base de données relationnelle, un *data frame* en R ou en Python-pandas. Cette classe peut être obtenue de sources ou types de données variés et aussi à partir d'un RDD. 

Le point d'entrée dans les fonctionnalités SQL en *Spark* est la classe *SQLContext*. 

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### 2.1 Inférer le schéma
La première opération consiste à construire le schéma des données. 

SparkSQL convertit en *data frame* un RDD composés d'objets *Row*. Cet objet est construit en passant une liste de (clef, valeur) comme [kwargs](http://deusyss.developpez.com/tutoriels/Python/args_kwargs/).  La clef définit le noim de colonne et le type (entier, flottant...) est déduit de la première ligne. Il est donc important qu'il n'y ait pas de données manquantes dans la première ligne du RDD.

Dans le cas de sdonnées de la KDD cup, il est nécessaire de scinder les lignes avec la "," comme séparateur et d'utiliser les informations décrivant les tâches pour extraire les noms de colonnes. 

In [ ]:
from pyspark.sql import Row
csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

Une fois le RDD créé par lignes, le schéma est inféré puis enregistré.

In [ ]:
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")

### 2.2 Exemple de requête SQL
Des requêtes SQL peuvent ensuite être exécutées.

In [ ]:
# Sélectionner les interactions "tcp" de plus de 1 s et sans transfert.
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

Les résultats d'une requête SQL sont des RDDs qui supportes les opérations sur les RDDs.

In [ ]:
# Sortie des durées avec les dst_bytes
tcp_interactions_out = tcp_interactions.map(lambda p: "Duration: {}, Dest. bytes: {}".format(p.duration, p.dst_bytes))
for ti_out in tcp_interactions_out.collect():
  print ti_out

Impression du schéma du *data frame*.

In [ ]:
interactions_df.printSchema()

## 3 Requêtes en tant qu'opérations sur DataFrame

SparkSQL inclut un langage pour la manipulation de données structurées. Il permet de combiner des méthodes de sélection, filtrage, regroupement... des données. L'exemple ci-dessous compte le nombre d'interactions par type de protocole. 

In [ ]:
from time import time

t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()
tt = time() - t0

print "Requete executee en {} secondes".format(round(tt,3))

Pour compter les interactions de moins d'une seconde sans transfert de données et groupés par protocole, il suffit d'ajouter des filtres. 

In [ ]:
t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").filter(interactions_df.duration>1000).filter(interactions_df.dst_bytes==0).groupBy("protocol_type").count().show()
tt = time() - t0

print "Requete executee en {} secondes".format(round(tt,3))

Ceci permet une approche exploratoire. Compte le nombre d'attaques et d'interactions "normales". Pour commencer, un label est ajouté.

In [ ]:
def get_label_type(label):
    if label!="normal.":
        return "attack"
    else:
        return "normal"
    
row_labeled_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5]),
    label=get_label_type(p[41])
    )
)
interactions_labeled_df = sqlContext.createDataFrame(row_labeled_data)

L'utilisation de l'interface de requête *Object Oriented* évite d'enregistrer le schéma.

Dénombrement des interactions "attaques" et "normales".L

In [ ]:
t0 = time()
interactions_labeled_df.select("label").groupBy("label").count().show()
tt = time() - t0

print "Requete executee en {} secondes".format(round(tt,3))

Dénombrement par label et type de protocole pour souligner le pouvoir discriminant de cette variable.

In [ ]:
t0 = time()
interactions_labeled_df.select("label", "protocol_type").groupBy("label", "protocol_type").count().show()
tt = time() - t0

print "Requete executee en {} secondes".format(round(tt,3))

Que dire du protocole `udp`?

Ajouter la prise en compte du transfert de données à partir de la cible.

In [ ]:
t0 = time()
interactions_labeled_df.select("label", "protocol_type", "dst_bytes").groupBy("label", "protocol_type", interactions_labeled_df.dst_bytes==0).count().show()
tt = time() - t0

print "Requete executee en {} secondes".format(round(tt,3))

Consulter les très nombreuses autres fonctionnalités, présentes ou à venir (version 2.0) dans la [documentation en ligne](http://spark.apache.org/docs/latest/sql-programming-guide.html).